In [17]:
import pandas as pd
import pickle
import re

In [ ]:
### Carga df desde excel
path_xlsx = "/Users/mac2/Library/CloudStorage/GoogleDrive-edwar.perez@vlex.com/Mi unidad/EP_Coding_Vlex/Martindale/datos_abogados.xlsx"
df_0 = pd.read_excel(path_xlsx)
df_0 = df_0.dropna(axis=1, how='all')
df_0.iloc[:,9:].head(2)
#df_0.head(2)


In [18]:
### Carga del df desde un archivo .pkl
with open("data_attornies.pkl", "rb") as f:
    data_for_PA = pickle.load(f) # Carga de lista de diccionarios de abogados de primera iteración, ya la lista no es 0.
    # Esta variable va a ser mas general por lo que se va a emplear mas adelante. o asignar a la primera corrida. 
data_for_PA
df = pd.DataFrame(data_for_PA)
df = df.where(pd.notnull(df), None)
df = df.dropna(axis=1, how='all')
df.drop(['Education & Credentials'], axis=1, inplace=True)
df.head(5)


,url_martindale,name,firm,url_firm_martindale,website_firm,Biography,Areas of Practice,Contact Information,University Attended,Law School Attended,...,"Other Practice Areas in Carolina, Puerto Rico","Other Practice Areas in Bayamon, Puerto Rico","Other Practice Areas in Dorado, Puerto Rico","Other Practice Areas in Guayama, Puerto Rico","Other Practice Areas in Isabela, Puerto Rico","Other Practice Areas in San German, Puerto Rico","Other Practice Areas in Vega Baja, Puerto Rico","Other Practice Areas in Cabo Rojo, Puerto Rico","Other Practice Areas in Salinas, Puerto Rico","Other Practice Areas in Aguadilla, Puerto Rico"
0,https://www.martindale.com/attorney/tessie-lea...,"Tessie Leal-Garabis, Esq.",Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Tessie Leal Garabis is a trial attorney with o...,"Trial Practice, Civil Appeals, Business Law, M...",787-650-6994Phone787-767-2645Faxwww.mellado.co...,"Bently University, B.A., 1995","Pontifical Catholic University of Puerto Rico,...",...,None,None,None,None,None,None,None,None,None,None
1,https://www.martindale.com/attorney/ramon-mell...,Ramón Mellado-González,Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Ramón has been an attorney and notary in Puert...,"Complex Business Litigation, Banks and Banking...",787-650-6994Phone787-767-2645Faxwww.mellado.co...,"University of Puerto Rico, B.A., Political Sci...","University of Puerto Rico, LL.B., 1958",...,None,None,None,None,None,None,None,None,None,None
2,https://www.martindale.com/attorney/edgardo-j-...,"Edgardo J. Hernández Ohárriz, Esq.","Hernandez - Oharriz & Santiago Law Firm, P.S.C.",https://www.martindale.com/organization/hernan...,http://www.hoslawfirm.com,Partner/co-founder of Hernandez-Oharriz & Sant...,"Personal Injury, Pedestrian Injuries, Plaintif...",855-789-8177Phone787-775-1672Faxwww.hoslawfirm...,"University of Puerto Rico, B.B.A., 2007; Unive...","Inter-American University, J.D., 2010",...,None,None,None,None,None,None,None,None,None,None
3,https://www.martindale.com/attorney/jaime-j-ap...,Jaime J. Aponte-Parsi,"Jaime Aponte-Parsi Law Offices, P.S.C.",https://www.martindale.com/organization/jaime-...,http://www.aponte-parsi.com,Former Partner: Maymi & Aponte-Parsi (2000-200...,"Corporate Law, Taxation, Real Estate, Estate S...",877-792-1046Phone787-622-7669Faxwww.aponte-par...,"Boston College, B.A., Economics and Political ...","Boston College, J.D., 1978",...,None,None,None,None,None,None,None,None,None,None
4,https://www.martindale.com/attorney/vicente-a-...,Vicente A. Sequeda-Torres,"Reichard & Calaf, P.S.C.",https://www.martindale.com/organization/reicha...,http://reichardcalaflaw.com,"Summer Program, Marshall Wythe School of Law, ...","Trusts and Estates, Real Estate, Trademark Inf...",866-677-2389Phone,"Marquette University, B.A., 1988","Inter-American University of Puerto Rico, J.D....",...,None,None,None,None,None,None,None,None,None,None


In [19]:
### Caso 1: Limpieza practice area
practice_cols = [col for col in df.columns if "practice area" in col.lower() and col != "Areas of Practice"]

# 2. Combinar columnas auxiliares en un string limpio
df['combined_practice'] = df[practice_cols].agg(
lambda row: '; '.join(
        str(val).strip()
        for val in row
        if val and str(val).strip().lower() != 'none'),
    axis=1)

# 3. Usar valor combinado solo si no está vacío
df['Areas of Practice'] = df.apply(
    lambda row: row['combined_practice'] if row['combined_practice'] else row['Areas of Practice'],
    axis=1)

# 4. Limpiar y estandarizar el texto final
df['Areas of Practice'] = (
    df['Areas of Practice']
    .fillna('')                                  # manejar posibles NaN
    .str.replace(',', ';')                       # reemplazar , por ;
    .str.replace(r'\s*;\s*', '; ', regex=True)   # limpiar espacios
    .str.strip('; ')                             # quitar ; sobrantes
    .apply(lambda x: '; ' + x if x and not x.startswith(';') else x))

# 5. Eliminar columnas auxiliares
df.drop(columns=practice_cols + ['combined_practice'], inplace=True)
df.head(20)

,url_martindale,name,firm,url_firm_martindale,website_firm,Biography,Areas of Practice,Contact Information,University Attended,Law School Attended,...,profile_peer_review_star,profile_peer_review_count,Minority Owned,Special Agencies,Military,Hobbies & interests,Transactions,Reported Cases,website,Affiliations
0,https://www.martindale.com/attorney/tessie-lea...,"Tessie Leal-Garabis, Esq.",Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Tessie Leal Garabis is a trial attorney with o...,; Trial Practice; Civil Appeals; Business Law;...,787-650-6994Phone787-767-2645Faxwww.mellado.co...,"Bently University, B.A., 1995","Pontifical Catholic University of Puerto Rico,...",...,None,None,None,None,None,None,None,None,None,None
1,https://www.martindale.com/attorney/ramon-mell...,Ramón Mellado-González,Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Ramón has been an attorney and notary in Puert...,; Complex Business Litigation; Banks and Banki...,787-650-6994Phone787-767-2645Faxwww.mellado.co...,"University of Puerto Rico, B.A., Political Sci...","University of Puerto Rico, LL.B., 1958",...,None,None,None,None,None,None,None,None,None,None
2,https://www.martindale.com/attorney/edgardo-j-...,"Edgardo J. Hernández Ohárriz, Esq.","Hernandez - Oharriz & Santiago Law Firm, P.S.C.",https://www.martindale.com/organization/hernan...,http://www.hoslawfirm.com,Partner/co-founder of Hernandez-Oharriz & Sant...,; Personal Injury; Pedestrian Injuries; Plaint...,855-789-8177Phone787-775-1672Faxwww.hoslawfirm...,"University of Puerto Rico, B.B.A., 2007; Unive...","Inter-American University, J.D., 2010",...,None,None,None,None,None,None,None,None,None,None
3,https://www.martindale.com/attorney/jaime-j-ap...,Jaime J. Aponte-Parsi,"Jaime Aponte-Parsi Law Offices, P.S.C.",https://www.martindale.com/organization/jaime-...,http://www.aponte-parsi.com,Former Partner: Maymi & Aponte-Parsi (2000-200...,; Corporate Law; Taxation; Real Estate; Estate...,877-792-1046Phone787-622-7669Faxwww.aponte-par...,"Boston College, B.A., Economics and Political ...","Boston College, J.D., 1978",...,None,None,None,None,None,None,None,None,None,None
4,https://www.martindale.com/attorney/vicente-a-...,Vicente A. Sequeda-Torres,"Reichard & Calaf, P.S.C.",https://www.martindale.com/organization/reicha...,http://reichardcalaflaw.com,"Summer Program, Marshall Wythe School of Law, ...",; Trusts and Estates; Real Estate; Trademark I...,866-677-2389Phone,"Marquette University, B.A., 1988","Inter-American University of Puerto Rico, J.D....",...,5.0,1,None,None,None,None,None,None,None,None
5,https://www.martindale.com/attorney/ross-alan-...,Ross Alan Miller,Gonzalez Olivieri LLC,https://www.martindale.com/organization/gonzal...,https://www.gonzalezolivierillc.com/,,; Immigration,346-299-8123Phonewww.gonzalezolivierillc.com,None,None,...,None,None,Yes,None,None,None,None,None,None,None
6,https://www.martindale.com/attorney/david-j-fe...,David J. Fernandez-Esteves,"Benjamin Acosta, Jr.",https://www.martindale.com/organization/benjam...,http://www.lobajr.com,None,,866-775-1143Phonewww.lobajr.com,"University of Puerto Rico at Cayey, B.A., cum ...","Inter-American University of Puerto Rico, J.D....",...,None,None,Yes,None,None,None,None,None,None,None
7,https://www.martindale.com/attorney/carlos-vil...,Carlos Villegas-Del Valle,"Fernández, Cuyar, Rovira & Plá, LLC",https://www.martindale.com/organization/fernan...,http://fclawpr.com,None,,939-296-3363Phonefclawpr.com/our-attorneys/car...,None,"Universidad de Puerto Rico, J.D.",...,None,None,Yes,None,None,None,None,None,None,None
8,https://www.martindale.com/attorney/hector-ped...,Héctor Pedrosa,Luis E. Miñana & Associates Abogados-Notarios,https://www.martindale.com/organization/luis-e...,http://securitiesatty.com,"Certified, Consumer Bankruptcy and Corporate B...",,787-303-0720Phone,"University of Puerto Rico, Río Piedras Campus,..."

In [5]:
### Caso 2: Limpieza contact info
list = df["Contact Information"].tolist()
list


['787-650-6994Phone787-767-2645Faxwww.mellado.com/tessie-leal-garabis',
 '787-650-6994Phone787-767-2645Faxwww.mellado.com/ramon-mellado-gonzalez',
 '855-789-8177Phone787-775-1672Faxwww.hoslawfirm.com',
 '877-792-1046Phone787-622-7669Faxwww.aponte-parsi.com',
 '866-677-2389Phone',
 '346-299-8123Phonewww.gonzalezolivierillc.com',
 '866-775-1143Phonewww.lobajr.com',
 '939-296-3363Phonefclawpr.com/our-attorneys/carlos-villegas',
 '787-303-0720Phone',
 '787-723-7735Fax787-725-4245Fax',
 '939-297-2014Phonewww.linkedin.com/in/helaine-gregory-68ba5ab',
 '215-791-8762PhoneStudentDisciplineDefense.com',
 '939-437-3015Phonewww.glennjameslaw.com/attorney-profile.html',
 '866-775-1143Phonewww.lobajr.com',
 '787-303-0720Phone787-761-5155Fax',
 '754-345-0953Phonewww.go-law.comwww.oneillgilmore.com',
 '787-723-7735Fax',
 '787-650-6994Phone787-767-2645Faxwww.mellado.com/hector-j-orejuela-davila',
 '346-299-8714Phone713-588-8683Faxwww.gonzalezolivierillc.comwww.gonzalezolivierillc.comevitaladeportacion.

In [20]:
def parse_contact_info(text):
    if not isinstance(text, str):
        return pd.Series([[], []], index=["Phones", "Faxes"])

    # Regex para teléfonos: busca 3 dígitos - 3 dígitos - 4 dígitos
    phone_fax_pattern = re.findall(r'(\d{3}-\d{3}-\d{4})(Phone|Fax)?', text)
    
    phones = []
    faxes = []

    # Clasificar como Phone o Fax si viene etiquetado
    i = 0
    while i < len(phone_fax_pattern):
        number, label = phone_fax_pattern[i]
        if label == 'Phone':
            phones.append(number)
        elif label == 'Fax':
            faxes.append(number)
        else:
            # Si no hay etiqueta, asumimos Phone si no está duplicado
            if number not in phones and number not in faxes:
                phones.append(number)
        i += 1

    # Regex para extraer links: comienzan por www. o contienen .com/.org/etc
    ##links = re.findall(r'(?:https?://)?(?:www\.)?[^\s]+(?:\.com|\.org|\.net|\.gov|\.edu|linkedin\.com)[^\s]*', text) Ya se obtuvieron en otra seccion

    return pd.Series([phones, faxes], index=["Phones", "Faxes"])

In [22]:
df[['Phones', 'Faxes']] = df['Contact Information'].apply(parse_contact_info)
df.drop('Contact Information', axis=1, inplace=True)
df.head(4)

,url_martindale,name,firm,url_firm_martindale,website_firm,Biography,Areas of Practice,University Attended,Law School Attended,Year of First Admission,...,Minority Owned,Special Agencies,Military,Hobbies & interests,Transactions,Reported Cases,website,Affiliations,Phones,Faxes
0,https://www.martindale.com/attorney/tessie-lea...,"Tessie Leal-Garabis, Esq.",Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Tessie Leal Garabis is a trial attorney with o...,; Trial Practice; Civil Appeals; Business Law;...,"Bently University, B.A., 1995","Pontifical Catholic University of Puerto Rico,...",1999,...,None,None,None,None,None,None,None,None,[787-650-6994],[787-767-2645]
1,https://www.martindale.com/attorney/ramon-mell...,Ramón Mellado-González,Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Ramón has been an attorney and notary in Puert...,; Complex Business Litigation; Banks and Banki...,"University of Puerto Rico, B.A., Political Sci...","University of Puerto Rico, LL.B., 1958",1958,...,None,None,None,None,None,None,None,None,[787-650-6994],[787-767-2645]
2,https://www.martindale.com/attorney/edgardo-j-...,"Edgardo J. Hernández Ohárriz, Esq.","Hernandez - Oharriz & Santiago Law Firm, P.S.C.",https://www.martindale.com/organization/hernan...,http://www.hoslawfirm.com,Partner/co-founder of Hernandez-Oharriz & Sant...,; Personal Injury; Pedestrian Injuries; Plaint...,"University of Puerto Rico, B.B.A., 2007; Unive...","Inter-American University, J.D., 2010",2011,...,None,None,None,None,None,None,None,None,[855-789-8177],[787-775-1672]
3,https://www.martindale.com/attorney/jaime-j-ap...,Jaime J. Aponte-Parsi,"Jaime Aponte-Parsi Law Offices, P.S.C.",https://www.martindale.com/organization/jaime-...,http://www.aponte-parsi.com,Former Partner: Maymi & Aponte-Parsi (2000-200...,; Corporate Law; Taxation; Real Estate; Estate...,"Boston College, B.A., Economics and Political ...","Boston College, J.D., 1978",1978,...,None,None,None,None,None,None,None,None,[877-792-1046],[787-622-7669]


In [23]:
df.to_excel("Martindale_attorneys_clean.xlsx",index=False)
df.head()

,url_martindale,name,firm,url_firm_martindale,website_firm,Biography,Areas of Practice,University Attended,Law School Attended,Year of First Admission,...,Minority Owned,Special Agencies,Military,Hobbies & interests,Transactions,Reported Cases,website,Affiliations,Phones,Faxes
0,https://www.martindale.com/attorney/tessie-lea...,"Tessie Leal-Garabis, Esq.",Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Tessie Leal Garabis is a trial attorney with o...,; Trial Practice; Civil Appeals; Business Law;...,"Bently University, B.A., 1995","Pontifical Catholic University of Puerto Rico,...",1999,...,None,None,None,None,None,None,None,None,[787-650-6994],[787-767-2645]
1,https://www.martindale.com/attorney/ramon-mell...,Ramón Mellado-González,Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Ramón has been an attorney and notary in Puert...,; Complex Business Litigation; Banks and Banki...,"University of Puerto Rico, B.A., Political Sci...","University of Puerto Rico, LL.B., 1958",1958,...,None,None,None,None,None,None,None,None,[787-650-6994],[787-767-2645]
2,https://www.martindale.com/attorney/edgardo-j-...,"Edgardo J. Hernández Ohárriz, Esq.","Hernandez - Oharriz & Santiago Law Firm, P.S.C.",https://www.martindale.com/organization/hernan...,http://www.hoslawfirm.com,Partner/co-founder of Hernandez-Oharriz & Sant...,; Personal Injury; Pedestrian Injuries; Plaint...,"University of Puerto Rico, B.B.A., 2007; Unive...","Inter-American University, J.D., 2010",2011,...,None,None,None,None,None,None,None,None,[855-789-8177],[787-775-1672]
3,https://www.martindale.com/attorney/jaime-j-ap...,Jaime J. Aponte-Parsi,"Jaime Aponte-Parsi Law Offices, P.S.C.",https://www.martindale.com/organization/jaime-...,http://www.aponte-parsi.com,Former Partner: Maymi & Aponte-Parsi (2000-200...,; Corporate Law; Taxation; Real Estate; Estate...,"Boston College, B.A., Economics and Political ...","Boston College, J.D., 1978",1978,...,None,None,None,None,None,None,None,None,[877-792-1046],[787-622-7669]
4,https://www.martindale.com/attorney/vicente-a-...,Vicente A. Sequeda-Torres,"Reichard & Calaf, P.S.C.",https://www.martindale.com/organization/reicha...,http://reichardcalaflaw.com,"Summer Program, Marshall Wythe School of Law, ...",; Trusts and Estates; Real Estate; Trademark I...,"Marquette University, B.A., 1988","Inter-American University of Puerto Rico, J.D....",1993,...,None,None,None,None,None,None,None,None,[866-677-2389],[]
